# LIMPIEZA Y ORGANIZACIÓN DEL DATA SET

#### 1. Importamos las librerias que nos seran utiles

In [2]:
import numpy as np
import pandas as pd
import re
import src.MetodosCleaningAnalysis as cf

#### 2. Importamos los datos

In [3]:
sharks = pd.read_csv('/home/julian/Cursos/Ironhack/Proyecto1/shark-data-cleaning/attacks.csv', encoding="ISO-8859-1")

In [4]:
sdf = pd.DataFrame(sharks)

In [5]:
sdf.shape

(25723, 24)

## HIPÓTESIS I

### ATAQUES MENSUALES

#### - Vamos a quedarnos con las columnas que serán relevantes para el analisis de nuestra primer hipotesis. Para ello, utilizaremos la función "keep_columns".

In [7]:
cf.keep_columns(['Case Number', 'Type', 'Case Number.1', 'Case Number.2'], sdf)

#### -Las columnas 'Case Number', 'Case Number.1' y 'Case Number.2' son relevantes para el analisis, sin embargo, vamos a chequear en que medida los valores de las columnas se parecen para prescindir de alguna de las tres. Para eso, aplicamos un método de numpy que nos devuelve un resultado booleano luego de comparar fila por fila si los valores de dos columnas coinciden o no.

In [8]:
compare1 = np.where(sdf['Case Number'] == sdf['Case Number.1'], True, False)

In [9]:
sdf['equal1'] = compare1

In [10]:
sdf['equal1'].value_counts() 

False    19445
True      6278
Name: equal1, dtype: int64

In [11]:
compare2 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)

In [12]:
sdf['equal2'] = compare2

In [13]:
sdf['equal2'].value_counts() 

False    19425
True      6298
Name: equal2, dtype: int64

In [14]:
compare3 = np.where(sdf['Case Number.1'] == sdf['Case Number.2'], True, False)

In [15]:
sdf['equal3'] = compare3

In [16]:
sdf['equal3'].value_counts() 

False    19441
True      6282
Name: equal3, dtype: int64

In [17]:
sdf.isnull().sum()

Case Number      17021
Type             19425
Case Number.1    19421
Case Number.2    19421
equal1               0
equal2               0
equal3               0
dtype: int64

#### - Al comparar las tres columnas entre sí, en donde menos diferencia de valores hay es entre 'Case Number' y 'Case Number .2'. De todas formas, para la cantidad de datos que tenemos la diferencia es mínima, y de las tres, la que menos cantidad de valores null tiene es 'Case Number', por lo tanto para obtener valores sobre la fecha de los ataques nos quedamos con ella, y con 'Type'.

In [18]:
cf.keep_columns(['Case Number', 'Type'], sdf)

#### -Vamos a eliminar los valores que no tengan formato de fecha, ya que nos interesa ver la estacionalidad mensual. Empezamos por los valores "NaN", y luego añadiremos una columna con los valores que solo. Para ello utilizaremos la funcion "filtración".

In [19]:
sdf.dropna(inplace=True)

In [20]:
sdf['Months'] = sdf['Case Number'].apply(cf.filtracion)

#### - Borramos los valores nulos que corresponden a los datos que no tenian registro mensual.

In [21]:
sdf.dropna(inplace=True)

#### - Exportamos el data frame a un archivo csv para luego realizar las visualizaciones.

In [22]:
sdf.to_csv('sdf',index=False)

#### - Ahora construimos un dataframe cuyas columnas contendrán las frecuencias mensuales de los ataques y los meses correspndientes, consideramos todos los tipos de ataque, con decenlace letal o no y renombramos las columnas para emprolijar.

In [23]:
ataques_mensuales = pd.DataFrame(sdf['Months'].value_counts()).reset_index()

In [24]:
c = {'Months': 'Monthly Attacks', 'index':'Month'}
ataques_mensuales = ataques_mensuales.rename(columns=c)

In [25]:
ataques_mensuales = ataques_mensuales.reindex([3, 11, 8, 5, 10, 4, 0, 1, 2, 7, 9, 6])

#### - Exportamos el data frame de ataques mensuales para utilizarlo en las visualizaciones

In [26]:
ataques_mensuales.to_csv('ataques_mensuales',index=False)

## HIPOTESIS II

### (ESPECIES/LETALIDAD)

In [27]:
sdf2= pd.DataFrame(sharks)

#### - Vamos a quedarnos con las columnas que nos interesan para el posterior analisis de nuestra segunda hipotesis: 'Species ' , 'Injury' y 'Fatal'. Para ello, utilizaremos la función "keep_columns". 

In [28]:
cf.keep_columns(['Injury', 'Fatal (Y/N)', 'Species '], sdf2)

#### - Procedemos a limpiar las col 'Injury' y 'Fatal',  si bien ambas columnas poseen datos sobre la letalidad, la columna 'Species' es la que mas datos nulos posee, y como nos interesa conocer las especies, eliminaremos las filas correspondientes a esos valores nulos.

In [29]:
sdf2['Fatal (Y/N)'].isna().value_counts()

True     19960
False     5763
Name: Fatal (Y/N), dtype: int64

In [30]:
sdf2['Injury'].isna().value_counts()

True     19449
False     6274
Name: Injury, dtype: int64

In [31]:
sdf2['Species '].isna().value_counts()

True     22259
False     3464
Name: Species , dtype: int64

In [32]:
sdf2.dropna(inplace=True)

In [33]:
sdf2['Identified Specie'] = sdf2['Species '].apply(cf.filtracion_especies)

#### - Vamos a construir dos data frames, uno con la cantidad de ataques por especie sin resultado letal, y otro igual, pero con las muertes:

##### 1. Ataques por especie

In [34]:
ataques = pd.DataFrame(sdf2[sdf2['Fatal (Y/N)'] == 'N'])

In [35]:
ataques = pd.DataFrame(sdf2['Identified Specie'].value_counts()).reset_index()

In [36]:
d = {'Identified Specie': 'Ataques', 'index':'Especie'}

In [37]:
ataques = ataques.rename(columns=d)

In [71]:
ataques.head(4)

,Especie,Ataques
0,white shark,625
1,tiger shark,256
2,bull shark,175
3,nurse shark,97


#### - Como hay gran cantidad de categorias de tiburones con pocos registros, vamos a considerar solo las que tengan al menos 15 registros de ataque.

In [38]:
ataques = ataques[ataques['Ataques'] > 15]

In [39]:
ataques.to_csv('Ataques-especies',index=False)

#### 2. Letalidad por especie

#### - Como la columna 'Fatal' posee más valores no nulos que 'Injury', potencialmente podríamos obtener más información sobre la letalidad de los ataques, vamos a prescindir de 'Injury' y vamos a quedarnos con los valores en donde hay un 'Y' en 'Fatal'.

In [40]:
sdf2.drop(columns=['Injury'], inplace=True)

In [41]:
letal = pd.DataFrame(sdf2[sdf2['Fatal (Y/N)'] == 'Y'])

In [42]:
letal2 = pd.DataFrame(letal['Identified Specie'].value_counts()).reset_index()

In [43]:
d = {'Identified Specie': 'Cantidad de Muertes', 'index':'Especie'}
letal2 = letal2.rename(columns=d)

In [70]:
letal2.head(4)

,Especie,Cantidad de Muertes
0,white shark,143
1,tiger shark,70
2,bull shark,37
3,zambesi shark,9


#### - Vamos a recortar el data frame para quedarnos con las especies que mas registros tienen

In [44]:
letal2.drop(range(8, 41), axis=0, inplace=True)

In [45]:
letal2.to_csv('especies-letalidad',index=False)

#### - Exportamos el data frame de ataques mensuales para utilizarlo en las visualizaciones

## HIPOTESIS III  

### (ACTIVIDADES/ATAQUES/MUERTES)

In [46]:
sdf3 = pd.DataFrame(sharks)

#### - Conservamos las columnas que nos interesan para comparar las Actividades que realizaban las víctimas y los ataques que resultaron en muerte. Se puede observar que la cantidad de nulos es similar para ambas variables, vamos a prescindir de ellos

In [47]:
cf.keep_columns(['Activity', 'Fatal (Y/N)'], sdf3)

In [48]:
sdf3['Fatal (Y/N)'].isna().value_counts()

True     19960
False     5763
Name: Fatal (Y/N), dtype: int64

In [49]:
sdf3['Activity'].isna().value_counts()

True     19965
False     5758
Name: Activity, dtype: int64

In [50]:
sdf3.dropna(inplace=True)

#### - Vamos a homogeneizar las columnas de actividades y fatalidades, usaremos la función "filtracion_actividades" y eliminaremos los valores nulos.

In [51]:
sdf3['Actividades'] = sdf3['Activity'].apply(cf.filtracion_actividades)

In [52]:
cf.keep_columns(['Actividades', 'Fatal (Y/N)'], sdf3)

In [53]:
sdf3.dropna(inplace=True)

#### - Definimos un data frame para registrar la cantidad de ataques no letales por actividad.

In [54]:
act = pd.DataFrame(sdf3[sdf3['Fatal (Y/N)'] == 'N'])

In [55]:
act = pd.DataFrame(sdf3['Actividades'].value_counts()).reset_index()

In [56]:
f = {'Actividades': 'Cantidad de Ataques', 'index':'Actividad'}

In [57]:
act = act.rename(columns=f)

In [58]:
act.head()

,Actividad,Cantidad de Ataques
0,surfing,1043
1,swimming,996
2,fishing,674
3,diving,471
4,spearfishing,379


#### - Vamos a quedarnos con las actividades que registren mas de 20 ataques

In [59]:
act = act[act['Cantidad de Ataques'] > 20]

#### - Ahora definimos un data frame para registrar la cantidad de muertes por actividad.

In [60]:
act_let = pd.DataFrame(sdf3[sdf3['Fatal (Y/N)'] == 'Y'])

In [61]:
act_let = pd.DataFrame(act_let['Actividades'].value_counts()).reset_index()

In [62]:
e = {'Actividades': 'Cantidad de Muertes', 'index':'Actividad'}

In [63]:
act_let = act_let.rename(columns=e)

In [72]:
act_let.head(4)

,Actividad,Cantidad de Muertes
0,swimming,409
1,diving,121
2,fishing,96
3,bathing,84


#### - Vamos a quedarnos con las actividades que registren mas de 20 muertes

In [64]:
act_let = act_let[act_let['Cantidad de Muertes'] > 20]

#### - Por último, vamos a unificar los dos data frames y lo exportaremos para visualizarlo.

In [65]:
j = act.set_index('Actividad').join(act_let.set_index('Actividad'))

In [66]:
j.dropna(inplace=True)

In [67]:
j.to_csv('actividades-ataques.csv')

In [73]:
j.head(4)

,Cantidad de Ataques,Cantidad de Muertes
Actividad,,
surfing,1043,60.0
swimming,996,409.0
fishing,674,96.0
diving,471,121.0
